In [ ]:
# Cell 1 - Imports and hyperparameters
import os
import time
import sys
import json
import pickle
import gc
from typing import List, Tuple, Dict
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (confusion_matrix, classification_report,
precision_score, recall_score, f1_score
)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, BackupAndRestore

LEARNING_RATE = 0.0001
EPOCHS = 500
BATCH_SIZE = 64

DATASET = "KTH"

MODEL_ARCH = "32"
USE_CONTEXT = True
USE_FOVEA   = True
PauseCounter = 500

CONTEXT_SHAPE = (32, 32, 1)
FOVEA_SHAPE   = (16, 16, 1)
REVERSE_RESOLUTION = False

ENABLE_XLA_JIT      = True
CACHE_TO_DISK       = True
PREFETCH_TO_DEVICE  = True
SHUFFLE_BUFFER      = 30000


In [ ]:
# Cell 2 - Configure TensorFlow JIT and GPU
try:
    tf.config.optimizer.set_jit(ENABLE_XLA_JIT)
    print("XLA JIT:", "ENABLED" if ENABLE_XLA_JIT else "DISABLED")
except Exception as e:
    print("Could not set XLA JIT:", e)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("GPU detected and configured for training:", gpus[0])
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Training will run on CPU.")


In [ ]:
# Cell 3 - Resolve streams, paths, classes, resolutions
BOTH_STREAMS = USE_CONTEXT and USE_FOVEA
if not (USE_CONTEXT or USE_FOVEA):
    raise ValueError("At least one of USE_CONTEXT or USE_FOVEA must be True.")

PREPROCESSED_DIR = f"/mnt/60FE87C2FE878F4A/Uni/Master's/Term2/Edge/Replication/datasets/{DATASET}/preprocessed_frames"

CLASS_NAMES = ["basketball","biking","diving",
               "golf_swing","horse_riding",
               "soccer_juggling","swing",
               "tennis_swing","trampoline_jumping",
               "volleyball_spiking","walking"] if DATASET == "UCF11" else ["boxing", "handclapping", "handwaving", "walking"]

NUM_CLASSES = len(CLASS_NAMES)

context_res = f"{CONTEXT_SHAPE[0]}x{CONTEXT_SHAPE[1]}"
fovea_res   = f"{FOVEA_SHAPE[0]}x{FOVEA_SHAPE[1]}"


In [ ]:
# Cell 4 - Define model architecture and build
if MODEL_ARCH == "64":
    CHANNEL1_SIZE = 16
    CHANNEL2_SIZE = 16
    DENSE1_SIZE = 10
    DENSE2_SIZE = 16
elif MODEL_ARCH == "32":
    CHANNEL1_SIZE = 16
    CHANNEL2_SIZE = 32
    DENSE1_SIZE = 20
    DENSE2_SIZE = 32
elif MODEL_ARCH == "main":
    CHANNEL1_SIZE = 8
    CHANNEL2_SIZE = 16
    DENSE1_SIZE = 8
    DENSE2_SIZE = 8

def build_branch(input_shape, name_prefix=""):
    inp = keras.Input(shape=input_shape, name=f"{name_prefix}input")
    x = layers.Conv2D(CHANNEL1_SIZE, kernel_size=(3, 3),
                      kernel_initializer="he_normal",
                      bias_initializer="zeros")(inp)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(CHANNEL2_SIZE, kernel_size=(3, 3),
                      kernel_initializer="he_normal",
                      bias_initializer="zeros")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    return inp, x

def build_model(
    use_context: bool,
    use_fovea: bool,
    context_input_shape=CONTEXT_SHAPE,
    fovea_input_shape=FOVEA_SHAPE,
    num_classes=NUM_CLASSES
):
    inputs = []
    branches = []
    if use_context:
        ctx_inp, ctx_out = build_branch(context_input_shape, name_prefix="context_")
        inputs.append(ctx_inp)
        branches.append(ctx_out)
    if use_fovea:
        fov_inp, fov_out = build_branch(fovea_input_shape, name_prefix="fovea_")
        inputs.append(fov_inp)
        branches.append(fov_out)
    if len(branches) == 1:
        fused = branches[0]
        model_name = "SingleStream_Context" if use_context and not use_fovea else "SingleStream_Fovea"
    else:
        fused = layers.Concatenate()(branches)
        model_name = "MultiResFusion"
    z = layers.Dense(DENSE1_SIZE, kernel_initializer="he_normal", bias_initializer="zeros")(fused)
    z = layers.Dropout(0.5)(z)
    z = layers.Dense(DENSE2_SIZE, kernel_initializer="he_normal", bias_initializer="zeros")(z)
    z = layers.Dropout(0.5)(z)
    output = layers.Dense(num_classes, activation="softmax",
                          kernel_initializer="he_normal",
                          bias_initializer="zeros")(z)
    model = Model(inputs=inputs, outputs=output, name=model_name)
    return model

model = build_model(USE_CONTEXT, USE_FOVEA, CONTEXT_SHAPE, FOVEA_SHAPE, NUM_CLASSES)
model.summary()


In [ ]:
# Cell 5 - Prepare output directories and paths
mode_tag = f"reverseResolution{MODEL_ARCH}" if REVERSE_RESOLUTION else f"both{MODEL_ARCH}" if (USE_CONTEXT and USE_FOVEA) else f"contextOnly{MODEL_ARCH}" if USE_CONTEXT else f"foveaOnly{MODEL_ARCH}" if USE_FOVEA else "invalid"
OUTPUT_DIR = f"{DATASET}_results/{mode_tag}_{EPOCHS}_{BATCH_SIZE}" \
             f"{'_ctx' + str(CONTEXT_SHAPE[0]) + 'x' + str(CONTEXT_SHAPE[1]) if USE_CONTEXT else ''}" \
             f"{'_fov' + str(FOVEA_SHAPE[0]) + 'x' + str(FOVEA_SHAPE[1]) if USE_FOVEA else ''}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CHECKPOINTS_DIR = os.path.join(OUTPUT_DIR, "checkpoints"); os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
PLOTS_DIR       = os.path.join(OUTPUT_DIR, "plots");       os.makedirs(PLOTS_DIR, exist_ok=True)
TEST_DATA_DIR   = os.path.join(OUTPUT_DIR, "test_data");   os.makedirs(TEST_DATA_DIR, exist_ok=True)
LOG_PATH        = os.path.join(OUTPUT_DIR, "training_metrics.csv")
PIPE_CACHE_DIR  = os.path.join(OUTPUT_DIR, "pipeline");     os.makedirs(PIPE_CACHE_DIR, exist_ok=True)


In [ ]:
# Cell 6 - tf.data utilities for loading images
def list_images_for_stream(split: str, resolution: str, class_names: List[str]) -> Tuple[List[str], np.ndarray, List[str]]:
    paths, labels, filenames = [], [], []
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(PREPROCESSED_DIR, split, resolution, class_name)
        if not os.path.isdir(class_dir):
            print("XXXX - Directory not found:", class_dir)
            continue
        for fname in sorted(os.listdir(class_dir)):
            if fname.lower().endswith('.png'):
                paths.append(os.path.join(class_dir, fname))
                labels.append(class_idx)
                filenames.append(fname)
    labels = np.array(labels, dtype=np.int32)
    return paths, labels, filenames

def make_tf_dataset(features_paths: Dict[str, List[str]],
                    labels: np.ndarray,
                    batch_size: int,
                    shuffle: bool,
                    drop_remainder: bool,
                    cache_file: str = None,
                    prefetch_to_device: bool = PREFETCH_TO_DEVICE):
    for k, v in features_paths.items():
        assert len(v) == len(labels), f"Length mismatch between {k} paths and labels."
    feat_elems = {k: tf.constant(v) for k, v in features_paths.items()}
    ds = tf.data.Dataset.from_tensor_slices((feat_elems, labels))
    opts = tf.data.Options()
    opts.experimental_deterministic = False
    opts.experimental_slack = True
    ds = ds.with_options(opts)
    if shuffle:
        ds = ds.shuffle(buffer_size=min(SHUFFLE_BUFFER, len(labels)))
    ctx_h, ctx_w, _ = CONTEXT_SHAPE
    fov_h, fov_w, _ = FOVEA_SHAPE
    def _loader(features, label):
        out_feats = {}
        if "context_input" in features:
            ctx = tf.io.read_file(features["context_input"])
            ctx = tf.io.decode_png(ctx, channels=1)
            ctx = tf.image.convert_image_dtype(ctx, tf.float32)
            ctx = tf.image.resize(ctx, [ctx_h, ctx_w])
            ctx = tf.ensure_shape(ctx, CONTEXT_SHAPE)
            out_feats["context_input"] = ctx
        if "fovea_input" in features:
            fov = tf.io.read_file(features["fovea_input"])
            fov = tf.io.decode_png(fov, channels=1)
            fov = tf.image.convert_image_dtype(fov, tf.float32)
            fov = tf.image.resize(fov, [fov_h, fov_w])
            fov = tf.ensure_shape(fov, FOVEA_SHAPE)
            out_feats["fovea_input"] = fov
        return out_feats, label
    ds = ds.map(_loader, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    if CACHE_TO_DISK and cache_file is not None:
        os.makedirs(os.path.dirname(cache_file), exist_ok=True)
        ds = ds.cache(cache_file)
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)
    if prefetch_to_device:
        try:
            if hasattr(tf.data, "experimental") and hasattr(tf.data.experimental, "prefetch_to_device"):
                ds = ds.apply(tf.data.experimental.prefetch_to_device("/GPU:0"))
            elif hasattr(tf.data, "experimental") and hasattr(tf.data.experimental, "copy_to_device"):
                ds = ds.apply(tf.data.experimental.copy_to_device("/GPU:0"))
        except Exception:
            pass
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [ ]:
# Cell 7 - Tee logger to mirror stdout to file
class Tee(object):
    def __init__(self, filename, mode="a"):
        self.file = open(filename, mode, encoding="utf-8")
        self.stdout = sys.stdout
    def write(self, data):
        self.file.write(data); self.file.flush()
        self.stdout.write(data); self.stdout.flush()
    def flush(self):
        self.file.flush(); self.stdout.flush()

tee = Tee(os.path.join(OUTPUT_DIR, "training_log.txt"), "a")
sys.stdout = tee
sys.stderr = tee


In [ ]:
# Cell 8 - Load dataset paths and prepare KFold inputs
print("# -------------------------------------------------------")
print("# Loading train, valid, test sets")
print("# Mode:", "Both streams" if (USE_CONTEXT and USE_FOVEA) else ("Context only" if USE_CONTEXT else "Fovea only"))
print("# -------------------------------------------------------")
print("# Using tf.data lazy pipeline (paths only, no big NumPy arrays)")

features_train = {}
labels_train = None

if USE_CONTEXT:
    ctx_train_paths, y_train_ctx, train_fns_ctx = list_images_for_stream('train', context_res, CLASS_NAMES)
    features_train["context_input"] = ctx_train_paths
    labels_train = y_train_ctx

if USE_FOVEA:
    fov_train_paths, y_train_fov, train_fns_fov = list_images_for_stream('train', f"centerCrop_{fovea_res}", CLASS_NAMES)
    features_train["fovea_input"] = fov_train_paths
    labels_train = y_train_fov if labels_train is None else labels_train
    if USE_CONTEXT:
        for i, (a, b) in enumerate(zip(train_fns_ctx, train_fns_fov)):
            if a != b:
                print(f"WARNING(train): filename mismatch at {i}: {a} vs {b}")
                break

features_valid = {}
if USE_CONTEXT:
    ctx_valid_paths, y_valid_ctx, valid_fns_ctx = list_images_for_stream('valid', context_res, CLASS_NAMES)
    features_valid["context_input"] = ctx_valid_paths
    y_valid_p = y_valid_ctx
if USE_FOVEA:
    fov_valid_paths, y_valid_fov, valid_fns_fov = list_images_for_stream('valid', f"centerCrop_{fovea_res}", CLASS_NAMES)
    features_valid["fovea_input"] = fov_valid_paths
    y_valid_p = y_valid_fov
    if USE_CONTEXT:
        for i, (a, b) in enumerate(zip(valid_fns_ctx, valid_fns_fov)):
            if a != b:
                print(f"WARNING(valid): filename mismatch at {i}: {a} vs {b}")
                break

features_test = {}
if USE_CONTEXT:
    ctx_test_paths, y_test_ctx, test_fns_ctx = list_images_for_stream('test', context_res, CLASS_NAMES)
    features_test["context_input"] = ctx_test_paths
    y_test_p = y_test_ctx
if USE_FOVEA:
    fov_test_paths, y_test_fov, test_fns_fov = list_images_for_stream('test', f"centerCrop_{fovea_res}", CLASS_NAMES)
    features_test["fovea_input"] = fov_test_paths
    y_test_p = y_test_fov
    if USE_CONTEXT:
        for i, (a, b) in enumerate(zip(test_fns_ctx, test_fns_fov)):
            if a != b:
                print(f"WARNING(test): filename mismatch at {i}: {a} vs {b}")
                break

X_train_val_features: Dict[str, List[str]] = {}
for k in features_train.keys():
    X_train_val_features[k] = features_train[k] + features_valid.get(k, []) + features_test.get(k, [])
y_train_val_labels = np.concatenate([labels_train, y_valid_p, y_test_p], axis=0)

print("Full training set (paths only) prepared:")
for k, v in X_train_val_features.items():
    print(f"  #{k} paths:", len(v))
print("  #labels:   ", len(y_train_val_labels))


In [ ]:
# Cell 9 - Set up KFold and training loop with checkpoints and plots
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies, fold_precisions, fold_recalls, fold_f1s = [], [], [], []
csv_logger = CSVLogger(LOG_PATH, append=True)
fold_classwise_reports = []
y_for_kf = y_train_val_labels

for fold_index, (train_idx, val_idx) in enumerate(kf.split(np.zeros_like(y_for_kf), y_for_kf), start=1):
    print(f"\n=== Fold {fold_index} ===")
    fold_metrics_path = os.path.join(PLOTS_DIR, f"fold_{fold_index}_metrics.json")
    if os.path.exists(fold_metrics_path):
        with open(fold_metrics_path, 'r') as f:
            metrics = json.load(f)
        print(f"[Fold {fold_index}] Training already completed. Resuming from saved metrics.")
        print(f"Fold {fold_index} - Validation Loss: {metrics['val_loss']:.4f}, Validation Accuracy: {metrics['val_acc']:.4f}")
        print(f"Fold {fold_index} - Weighted Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, F1: {metrics['f1']:.4f}")
        fold_accuracies.append(metrics['val_acc'])
        fold_precisions.append(metrics['precision'])
        fold_recalls.append(metrics['recall'])
        fold_f1s.append(metrics['f1'])
        fold_classwise_reports.append(metrics.get("class_report", {}))
        continue

    def slice_by_idx(lst, idxs): return [lst[i] for i in idxs]

    features_train_fold: Dict[str, List[str]] = {}
    features_val_fold:   Dict[str, List[str]] = {}
    for k, v in X_train_val_features.items():
        features_train_fold[k] = slice_by_idx(v, train_idx)
        features_val_fold[k]   = slice_by_idx(v, val_idx)

    y_train_fold = y_for_kf[train_idx]
    y_val_fold   = y_for_kf[val_idx]

    cache_train     = os.path.join(PIPE_CACHE_DIR, f"fold{fold_index}_train.cache") if CACHE_TO_DISK else None
    cache_val_train = os.path.join(PIPE_CACHE_DIR, f"fold{fold_index}_val_train.cache") if CACHE_TO_DISK else None
    cache_val_eval  = os.path.join(PIPE_CACHE_DIR, f"fold{fold_index}_val_eval.cache") if CACHE_TO_DISK else None

    ds_train     = make_tf_dataset(features_train_fold, y_train_fold,
                                    batch_size=BATCH_SIZE, shuffle=True,  drop_remainder=True,
                                    cache_file=cache_train)
    ds_val_train = make_tf_dataset(features_val_fold,   y_val_fold,
                                    batch_size=BATCH_SIZE, shuffle=False, drop_remainder=True,
                                    cache_file=cache_val_train)
    ds_val_eval  = make_tf_dataset(features_val_fold,   y_val_fold,
                                    batch_size=BATCH_SIZE, shuffle=False, drop_remainder=False,
                                    cache_file=cache_val_eval)
    
    model = build_model(USE_CONTEXT, USE_FOVEA, CONTEXT_SHAPE, FOVEA_SHAPE, NUM_CLASSES)
    opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=opt,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=["accuracy"],
        jit_compile=False
    )

    fold_best_checkpoint_path = os.path.join(CHECKPOINTS_DIR, f"fold_{fold_index}_best_model.keras")
    fold_backup_dir = os.path.join(CHECKPOINTS_DIR, f"fold_{fold_index}_backup")
    if os.path.exists(fold_backup_dir) and os.listdir(fold_backup_dir):
        print(f"[Fold {fold_index}] Found an existing backup. Training will resume from the last epoch.")
    else:
        print(f"[Fold {fold_index}] No existing backup. Training will start from scratch.")
    os.makedirs(fold_backup_dir, exist_ok=True)

    best_checkpoint_callback = ModelCheckpoint(
        filepath=fold_best_checkpoint_path, monitor='val_loss',
        save_best_only=True, mode='min', verbose=1
    )
    backup_and_restore_callback = BackupAndRestore(backup_dir=fold_backup_dir)

    class PauseEveryN(tf.keras.callbacks.Callback):
        def __init__(self, n=100, seconds=300):
            super().__init__(); self.n = n; self.seconds = seconds
        def on_epoch_end(self, epoch, logs=None):
            if (epoch + 1) % self.n == 0:
                print(f"\n[PauseEveryN] Sleeping {self.seconds} seconds after epoch {epoch+1} ...")
                time.sleep(self.seconds)
                
    pause_cb = PauseEveryN(n=PauseCounter, seconds=120)

    history = model.fit(
        x=ds_train,
        epochs=EPOCHS,
        validation_data=ds_val_train,
        verbose=2,
        callbacks=[
            best_checkpoint_callback,
            csv_logger,
            backup_and_restore_callback,
            pause_cb
        ]
    )

    model.load_weights(fold_best_checkpoint_path)
    val_loss, val_acc = model.evaluate(ds_val_eval, verbose=0)
    print(f"Fold {fold_index} - Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    y_val_pred_probs = model.predict(ds_val_eval, verbose=0)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)

    precision = precision_score(y_val_fold, y_val_pred, average="weighted", zero_division=0)
    recall    = recall_score(y_val_fold, y_val_pred, average="weighted", zero_division=0)
    f1        = f1_score(y_val_fold, y_val_pred, average="weighted", zero_division=0)
    print(f"Fold {fold_index} - Weighted Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    fold_accuracies.append(val_acc); fold_precisions.append(precision)
    fold_recalls.append(recall);     fold_f1s.append(f1)

    test_data_npz_path = os.path.join(TEST_DATA_DIR, f"fold_{fold_index}_test_data.npz")
    npz_kwargs = {"y": y_val_fold}
    if USE_CONTEXT:
        npz_kwargs["X_context_paths"] = np.array(features_val_fold["context_input"], dtype=object)
    if USE_FOVEA:
        npz_kwargs["X_fovea_paths"]   = np.array(features_val_fold["fovea_input"], dtype=object)
    np.savez_compressed(test_data_npz_path, **npz_kwargs)
    print(f"[Fold {fold_index}] Test data saved to {test_data_npz_path}")

    cm = confusion_matrix(y_val_fold, y_val_pred)
    cm_df = pd.DataFrame(cm, index=CLASS_NAMES, columns=CLASS_NAMES)
    cm_csv_path  = os.path.join(PLOTS_DIR, f"fold_{fold_index}_confusion_matrix.csv")
    cm_plot_path = os.path.join(PLOTS_DIR, f"fold_{fold_index}_confusion_matrix.png")
    cm_df.to_csv(cm_csv_path, index=True)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Fold {fold_index} - Confusion Matrix")
    plt.ylabel('Predicted Label'); plt.xlabel('True Label')
    plt.savefig(cm_plot_path); plt.close()

    class_report_dict = classification_report(
        y_val_fold, y_val_pred, target_names=CLASS_NAMES, digits=4, output_dict=True, zero_division=0
    )
    per_class_dict = {cls: {
        "precision": class_report_dict[cls]["precision"] * 100.0,
        "recall":    class_report_dict[cls]["recall"]    * 100.0,
        "f1":        class_report_dict[cls]["f1-score"]  * 100.0
    } for cls in CLASS_NAMES}
    fold_classwise_reports.append(per_class_dict)

    shape_tag = []
    if USE_CONTEXT: shape_tag.append(f"ctx{CONTEXT_SHAPE[0]}x{CONTEXT_SHAPE[1]}")
    if USE_FOVEA:   shape_tag.append(f"fov{FOVEA_SHAPE[0]}x{FOVEA_SHAPE[1]}")
    shape_tag = "_".join(shape_tag) if shape_tag else "unknown"
    history_pickle_path = os.path.join(PLOTS_DIR, f"history_{mode_tag}_{shape_tag}_fold{fold_index}.pkl")
    history_json_path   = os.path.join(PLOTS_DIR, f"history_{mode_tag}_{shape_tag}_fold{fold_index}.json")
    with open(history_pickle_path, 'wb') as f: pickle.dump(history.history, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(history_json_path, 'w') as f: json.dump(history.history, f)

    plt.figure(figsize=(8, 6)); plt.plot(history.history['accuracy'], label='Train Accuracy'); plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'Fold {fold_index} Accuracy'); plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(PLOTS_DIR, f"fold_{fold_index}_accuracy.png")); plt.close()

    plt.figure(figsize=(8, 6)); plt.plot(history.history['loss'], label='Train Loss'); plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'Fold {fold_index} Loss'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(PLOTS_DIR, f"fold_{fold_index}_loss.png")); plt.close()

    fold_metrics = {"val_loss": val_loss, "val_acc": val_acc, "precision": precision, "recall": recall, "f1": f1, "class_report": per_class_dict}
    with open(fold_metrics_path, 'w') as f: json.dump(fold_metrics, f)

    try:
        del ds_train, ds_val_train, ds_val_eval, features_train_fold, features_val_fold
    except Exception:
        pass
    try:
        del y_train_fold, y_val_fold, y_val_pred_probs, y_val_pred, cm, cm_df, class_report_dict, per_class_dict
    except Exception:
        pass
    tf.keras.backend.clear_session()
    gc.collect()


In [ ]:
# Cell 10 - Aggregate cross-validation metrics and save per-class averages
mean_acc = np.mean(fold_accuracies)
mean_pre = np.mean(fold_precisions)
mean_rec = np.mean(fold_recalls)
mean_f1  = np.mean(fold_f1s)

print("\n=== 5-Fold Cross-Validation Results (Weighted Average) ===")
print(f"Accuracy:  {mean_acc:.4f}")
print(f"Precision: {mean_pre:.4f}")
print(f"Recall:    {mean_rec:.4f}")
print(f"F1-score:  {mean_f1:.4f}")

def average_class_metrics(list_of_dicts, class_names):
    avg_dict = {}
    for cls_name in class_names:
        precs = [d[cls_name]["precision"] for d in list_of_dicts]
        recs  = [d[cls_name]["recall"]   for d in list_of_dicts]
        f1s   = [d[cls_name]["f1"]       for d in list_of_dicts]
        avg_dict[cls_name] = {
            "precision": np.mean(precs),
            "recall":    np.mean(recs),
            "f1":        np.mean(f1s)
        }
    return avg_dict

avg_classwise = average_class_metrics(fold_classwise_reports, CLASS_NAMES)

print("\n=== Table 1 (Per-Class Averages over 5-Fold) ===")
for cls_name in CLASS_NAMES:
    metrics = avg_classwise[cls_name]
    print(f"{cls_name} => Precision: {metrics['precision']:.2f}%, "
          f"Recall: {metrics['recall']:.2f}%, F1: {metrics['f1']:.2f}%")

output_file = os.path.join(PLOTS_DIR, "per_class_metrics.txt")
with open(output_file, "w") as f:
    f.write("\n=== Table 1 (Per-Class Averages over 5-Fold) ===\n")
    for cls_name in CLASS_NAMES:
        metrics = avg_classwise[cls_name]
        line = (f"{cls_name} => Precision: {metrics['precision']:.2f}%, "
                f"Recall: {metrics['recall']:.2f}%, F1: {metrics['f1']:.2f}%\n")
        f.write(line)
print(f"Results saved to {output_file}")


In [ ]:
# Cell 11 - TFLite conversion and C array export for all folds
import subprocess
import re

CHECKPOINTS_DIR = os.path.join(OUTPUT_DIR, "checkpoints")
TFLITE_DIR = os.path.join(OUTPUT_DIR, "quantized_models_ptq")
C_ARRAY_DIR = os.path.join(OUTPUT_DIR, "c_arrays_ptq")

os.makedirs(TFLITE_DIR, exist_ok=True)
os.makedirs(C_ARRAY_DIR, exist_ok=True)

def quantize_and_convert_to_c(fold_index):
    keras_model_path = os.path.join(CHECKPOINTS_DIR, f"fold_{fold_index}_best_model.keras")
    tflite_model_path = os.path.join(TFLITE_DIR, f"fold{fold_index}.tflite")
    tmp_c_path = os.path.join(C_ARRAY_DIR, f"_tmp_fold{fold_index}.h")
    final_c_path = os.path.join(C_ARRAY_DIR, f"fold{fold_index}_model_data.h")
    var_name = f"fold{fold_index}_model"
    model = tf.keras.models.load_model(keras_model_path)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open(tflite_model_path, "wb") as f:
        f.write(tflite_model)
    print(f"✅ TFLite model saved: {tflite_model_path}")
    with open(tmp_c_path, "w") as out:
        subprocess.run(["xxd", "-i", tflite_model_path], stdout=out)
    with open(tmp_c_path, "r") as f:
        content = f.read()
    content = re.sub(r'unsigned char\s+\w+\s*\[\]', f'unsigned char {var_name}[]', content)
    content = re.sub(r'unsigned int\s+\w+_len', f'unsigned int {var_name}_len', content)
    guard_name = f"FOLD{fold_index}_MODEL_DATA_H"
    header_guard = f"#ifndef {guard_name}\n#define {guard_name}\n\n"
    footer_guard = "\n#endif\n"
    with open(final_c_path, "w") as f:
        f.write(header_guard)
        f.write(content)
        f.write(footer_guard)
    os.remove(tmp_c_path)
    print(f"✅ C array saved with variable: {var_name} in {final_c_path}")

for fold in range(1, 6):
    quantize_and_convert_to_c(fold)
